In [1]:
# Importing findspark and initialize Spark session
import findspark

In [2]:
findspark.init('/Users/DoddiC/Downloads/spark-3.4.0-bin-hadoop3')

In [4]:
import pyspark #because we imported findspark

from pyspark.sql import SparkSession

# Dummy Testing Code

# Create a sample DataFrame
# data = [("Alice", 25), ("Bob", 30), ("Charlie", 28)]
# columns = ["Name", "Age"]
# sample_df = spark.createDataFrame(data, columns)

# Show the sample DataFrame
# sample_df.show()

# Perform a simple operation: Select and display the names of individuals older than 28
# result_df = sample_df.filter(sample_df["Age"] > 28).select("Name")
# result_df.show()

# adding gracefullyshutdown = true for graceful shutdown of processes (running) and getORCreate for reusing a Spark session if it's already running, reducing overhead
spark = SparkSession \
    .builder \
    .appName("Streaming from Kafka") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0') \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local[*]") \ 
    .getOrCreate()

# spark.stop()


In [5]:
# Import required modules
import sys
from pyspark.sql import *
from pyspark.sql.types import StructType,StructField 
from pyspark.sql.types import StringType, IntegerType, ArrayType
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [6]:
#from pyspark.sql import SparkSession

# Psuedo: 
# Goal: Read data from Kafka topic
# 1. Specify data format
# 2. List Kafka broker addresses
# 3. Subscribe to the specified Kafka topic
# 4. Set a starting offset to the earliest available
# 5. loading the Kafka data into a dataframe (might just be .load())
df = spark \
    .read.format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "first_topic") \
    .option("startingOffsets", "earliest") \
    .load() \


In [7]:
df.show()

# Seeing if i can edit the default limit (20)
# spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
# spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 50)

23/06/04 16:57:32 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/06/04 16:57:32 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/06/04 16:57:32 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/06/04 16:57:32 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
23/06/04 16:57:32 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+-------+--------------------+-----------+---------+------+--------------------+-------------+
|    key|               value|      topic|partition|offset|           timestamp|timestampType|
+-------+--------------------+-----------+---------+------+--------------------+-------------+
|   [30]|[7B 22 69 64 22 3...|first_topic|        0|     0|2023-06-04 15:53:...|            0|
|   [31]|[7B 22 69 64 22 3...|first_topic|        0|     1|2023-06-04 15:53:...|            0|
|   [32]|[7B 22 69 64 22 3...|first_topic|        0|     2|2023-06-04 15:53:...|            0|
|   [33]|[7B 22 69 64 22 3...|first_topic|        0|     3|2023-06-04 15:53:...|            0|
|   [34]|[7B 22 69 64 22 3...|first_topic|        0|     4|2023-06-04 15:53:...|            0|
|   [35]|[7B 22 69 64 22 3...|first_topic|        0|     5|2023-06-04 15:53:...|            0|
|   [36]|[7B 22 69 64 22 3...|first_topic|        0|     6|2023-06-04 15:53:...|            0|
|   [37]|[7B 22 69 64 22 3...|first_topic|        

In [8]:
# df = df.withColumn("json", from_json(df["json"], jsonSchema)) \
#                 .select("json.*")

# Parse value from binay to string
json_df = df.selectExpr("cast(value as string) as value")

# # Apply Schema to JSON value column and expand the value
# from pyspark.sql.functions import from_json

# json_expanded_df = json_df.withColumn("value", from_json(json_df["value"], json_schema)).select("value.*") 

In [9]:
#jsonSchema from Pyspark

jsonSchema = StructType([StructField("id", StringType(), True), StructField("Year", StringType(), True),
                                     StructField("Reporting_Airline", StringType(), True)])

In [10]:

json_expanded_df = json_df.withColumn("value", from_json(json_df["value"], jsonSchema)).select("value.*") 

In [11]:
json_expanded_df.show()

23/06/04 16:58:17 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/06/04 16:58:17 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/06/04 16:58:17 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/06/04 16:58:17 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
23/06/04 16:58:17 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+---+----+-----------------+
| id|Year|Reporting_Airline|
+---+----+-----------------+
|  0|1998|               NW|
|  1|2009|               FL|
|  2|2013|               MQ|
|  3|2010|               DL|
|  4|2006|               US|
|  5|1995|               DL|
|  6|2006|               CO|
|  7|2019|               9E|
|  8|2008|               YV|
|  9|2018|               WN|
| 10|1991|               US|
| 11|2014|               WN|
| 12|1994|               AA|
| 13|2013|               OO|
| 14|2003|               UA|
| 15|1988|               PI|
| 16|2007|               NW|
| 17|2015|               AS|
| 18|2006|               UA|
| 19|2017|               WN|
+---+----+-----------------+
only showing top 20 rows



In [12]:
json_expanded_df.select('Reporting_Airline').groupBy('Reporting_Airline').count().show()

23/06/04 16:58:42 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/06/04 16:58:42 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/06/04 16:58:42 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/06/04 16:58:42 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
23/06/04 16:58:42 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+-----------------+-----+
|Reporting_Airline|count|
+-----------------+-----+
|               US|   40|
|               YV|    6|
|               WN|   72|
|               PI|    4|
|               EV|   11|
|               HP|   11|
|               NW|   25|
|               MQ|   22|
|               DL|   77|
|               OO|   27|
|               UA|   39|
|               XE|    8|
|               EA|    1|
|               YX|    2|
|               TW|    9|
|               B6|    7|
|               F9|    4|
|               PS|    1|
|               FL|    5|
|               CO|   31|
+-----------------+-----+
only showing top 20 rows



In [13]:
df_filtered = json_expanded_df.select('id','Year','Reporting_Airline').filter('Year >= 2015')

json_expanded_df.createOrReplaceTempView("json_expanded_table")

query = """
    SELECT id, Year, Reporting_Airline
    FROM json_expanded_table
    WHERE Year >= 2015
"""

df_filtered = spark.sql(query)